# Universe creation and manipulation

Using MDAnalysis, RDKit, and OpenMM we will create an alpha helix from a sequence of amino acids. There are better tools to do that specific task, but it will make us use various interesting features of MDAnalysis.

In [ ]:
import MDAnalysis as mda
import nglview as nv

In [ ]:
from simtk import openmm, unit
from simtk.openmm import app
import pdbfixer

In [ ]:
import rdkit

In [ ]:
import numpy as np

## Creating a topology from a sequence using RDKit

RDKit can create a peptide from a sequence using [`MolFromSequence`](https://www.rdkit.org/docs/source/rdkit.Chem.rdmolfiles.html#rdkit.Chem.rdmolfiles.MolFromSequence). Create an RDKit molecule for the [WALP peptide](https://en.wikipedia.org/wiki/WALP_peptide) with the sequence GWWLALALALALALALWWA.

In [ ]:
peptide = rdkit.Chem.MolFromSequence('GWWLALALALALALALWWA')
peptide

Our peptide has a topology but no coordinates. Ideally, we would create 3D coordinates using [`AllChem.EmbedMolecule`](https://www.rdkit.org/docs/source/rdkit.Chem.rdDistGeom.html?highlight=embedmolecule#rdkit.Chem.rdDistGeom.EmbedMolecule), sadly the method fails with our molecule. Instead, generate 2D coordinates using [`Compute2DCoords`](https://www.rdkit.org/docs/source/rdkit.Chem.rdchem.html?highlight=compute2dcoord#rdkit.Chem.rdchem.Mol.Compute2DCoords).

In [ ]:
peptide.Compute2DCoords()

## Fix the backbone with MDAnalysis

At this point we have the correct topology, but the coordinates of the atoms are completely wrong. We will first fold the backbone into an helix using MDAnalysis, then we will fix the side chains using OpenMM.

### Loading the structure in MDAnalysis

MDAnalysis tries to be interoperable with as many tools as possible. Since version 2.0.0, it is possible to create a universe from an RDKit molecule, and vice versa.

Create a universe `u` from our RDKit peptide, then visualise the universe using NGLview.

In [ ]:
u = mda.Universe(peptide)

In [ ]:
v = nv.show_mdanalysis(u)
v.add_ball_and_stick()
v

### Finding the phi angles

To create an alpha helix, we need to set the phi, psi, and omega dihedral angles to -60°, -50°, and 180°, respectively. The phi angle for a given residue is defined by the atoms `C` of the previous residue, `N`, `CA`, and `C`. The psi angle is defined by the atoms `N`, `C`, `CA`, and `N` from the next residue. Omega is defined by atoms `CA` and `C` of the previous residue, `N`, and `CA`.

Before we can set the dihedral angles, we need to find them and get their current values. For each residue, create an atom group containing the 4 defining atoms of the phi angle.

In [ ]:
all_phi = u.residues.phi_selections()
all_phi

Calculate the angle in degrees for each phi angle.

In [ ]:
[ag.dihedral.value() for ag in u.residues.phi_selections() if ag is not None]

### Rotating around a bond

To fold the backbone, we will rotate the structure around the appropriate bonds to set the dihedral angles to the expected value. Let's start with just one dihedral.

To rotate around a bond, we need to:
* get the bond vector around which to do the rotation
* get the rotation center
* identify what are the atoms to move

For one of the dihedral angles of one residue, find the bond around which to rotate and compute its vector.

In [ ]:
dihedral_selection = all_phi[4]
# The bonds within the selection are C-N, N-CA, CA-C.
# The second one is the one we are interested in.
central_bond = dihedral_selection.intra_bonds[1]
bond_vector = central_bond.atoms[1].position - central_bond.atoms[0].position

We will set the mobile part of the molecule as what comes *after* the bond. The centre of rotation is the atom that is part of the bond and part of the mobile part. Extract the coordinates of the center.

In [ ]:
centre = central_bond.atoms[1].position

Select the mobile part of the molecule. Here, we can assume that the atoms are ordered such as all the atoms after the bond will move with the rotation.

In [ ]:
mobile = u.atoms[central_bond.atoms[1].ix:]

To rotate the atoms, use the [`rotateby`](https://docs.mdanalysis.org/2.0.0-dev0/documentation_pages/core/groups.html?#MDAnalysis.core.groups.AtomGroup.rotateby) method of `AtomGroup`; then look at the structure with NGLview.

In [ ]:
mobile.rotateby(-60, bond_vector, centre)

In [ ]:
v = nv.show_mdanalysis(u)
v.add_ball_and_stick()
v

### Building the helix

Now repeat that operation for all dihedral angles.

In [ ]:
def set_dihedrals(ag, target_dihedral, dihedral_name):
    all_atoms = ag.universe.atoms
    
    for residue in ag.residues:
        if dihedral_name == 'phi':
            dihedral_selection = residue.phi_selection()
        elif dihedral_name == 'psi':
            dihedral_selection = residue.psi_selection()
        elif dihedral_name == 'omega':
            dihedral_selection = residue.omega_selection()
        else:
            raise ValueError(f'dihedral_name cannot be "{dihedral_name}"')

        if dihedral_selection is None:
            continue

        central_bond = dihedral_selection.intra_bonds[1]
        bond_vector = central_bond.atoms[1].position - central_bond.atoms[0].position
        centre = central_bond.atoms[1].position

        mobile = all_atoms[central_bond.atoms[1].ix:] & ag
        
        dihedral = dihedral_selection.dihedral.value()
        dihedral_delta = target_dihedral - dihedral

        mobile.rotateby(dihedral_delta, bond_vector, centre)

In [ ]:
set_dihedrals(u.atoms, -60, 'phi')

In [ ]:
set_dihedrals(u.atoms, -50, 'psi')

In [ ]:
set_dihedrals(u.atoms, 180, 'omega')

How does the structure look like now?

In [ ]:
v = nv.show_mdanalysis(u)
v.add_ball_and_stick()
v

## Fix the side chains with OpenMM

The backbone of the peptide is folded as an helix. However the side chains are completely unphysical. To fix this we will use OpenMM to minimize the energy of the system. This requires to setup a simulation system so, first, we will have to add the hydrogen atoms.

### Setting a box size

We will be able to set the box when builing the OpenMM system. But this is an MDAnalysis workshop so let's do it with MDAnalysis!

What is the current box size?

In [ ]:
u.dimensions

The `dimension` array contains the length of the box vectors and the box vector angles.

We will use an implicit solvent to minimise the energy of our peptide, we could also have done it in vacuum, this means the size of the box does not matter much as long as it is large enough for the peptide not to interact with its periodic image. Set the box for this universe to be cubic with 10 nm sides.

In [ ]:
u.dimensions = [100, 100, 100, 90, 90, 90]

### Adding the hydrogen atoms

Our peptide does not have any hydrogen atoms. We could have added them earlier with RDKit; instead we will add them using [pdbfixer](https://github.com/openmm/pdbfixer), a tool related to OpenMM.

Sadly, we cannot create an OpenMM topology from MDAnalysis, yet. To pass our system from MDAnalysis to OpenMM write it as a PDB file named `walp_broken.pdb`.

In [ ]:
u.atoms.write('walp_broken.pdb')

Now, we can load the PDB in pdbfixer and add the hydrogen atoms. Here is the code to do so.

In [ ]:
u.atoms.write('walp_broken.pdb')

In [ ]:
# Load the PDB file we just created
fixer = pdbfixer.PDBFixer('walp_broken.pdb')
# The PDBFixer object first need to find what is missing
# in the structure and we need to request it explicitly
# to do so.
fixer.findMissingResidues()
fixer.findMissingAtoms()
# Finally! We can add the hydrogen atoms, both in the
# topology and the structure.
fixer.addMissingHydrogens()

### Loading the structure with hydrogen atoms in MDAnalysis

MDAnalysis can create a Universe from various OpenMM object including [`Topology`](http://docs.openmm.org/7.0.0/api-python/generated/simtk.openmm.app.topology.Topology.html), [`PDBFile`](http://docs.openmm.org/7.0.0/api-python/generated/simtk.openmm.app.pdbfile.PDBFile.html#), [`PDBxFile`](http://docs.openmm.org/7.0.0/api-python/generated/simtk.openmm.app.pdbxfile.PDBxFile.html#), [`Modeller`](http://docs.openmm.org/7.0.0/api-python/generated/simtk.openmm.app.modeller.Modeller.html#), and [`Simulation`](http://docs.openmm.org/7.0.0/api-python/generated/simtk.openmm.app.simulation.Simulation.html#). It cannot, however, create a Universe from a PDBFixer object. Though, we can create a Universe from an array of coordinate and the topology in the PDBFixer object!

The coordinates of the atoms are accessible as `fixer.positions`. Create an numpy array of these positions. Be carefull that pdbfixer uses nanometers by default, while MDAnalysis works in ångströms.

In [ ]:
position_array = np.array(fixer.positions.value_in_unit(unit.angstrom))

Now, create an MDAnalysis's Universe from the `fixer.topology` and the array of coordinates you just created. The topology in a PDBFixer object is not exactly an OpenMM topology (it does not use the same python class), but it uses the same interface making it compatible. Because it is not the same class as an OpenMM topology, MDAnalysis does not automatically recognise how to read it; use `topology_format='OPENMMTOPOLOGY'` to specify a topology parser.

In [ ]:
u_hydrogen = mda.Universe(fixer.topology, position_array, topology_format='OPENMMTOPOLOGY')

Visualise the new universe. Does it contain hydrogen atoms?

In [ ]:
v = nv.show_mdanalysis(u_hydrogen)
v.add_ball_and_stick()
v

### Buiding the simulation for the energy minimisation

Everything is finally ready to setup OpenMM. We will use the Amber10 force field and an implicit solvent.

In [ ]:
forcefield = app.ForceField('amber10.xml', 'amber10_obc.xml')
system = forcefield.createSystem(fixer.topology)
# We do not need an integrator, but OpenMM will not create a
# simulation system without one.
integrator = openmm.LangevinIntegrator(300 * unit.kelvin, 1 / unit.picosecond, 2 * unit.femtosecond)

We spent some time folding the backbone, we do not want the energy minimisation to alter it. We will apply position restraints on the "N", "C", and "CA" atoms of the backbone.

In [ ]:
force = openmm.CustomExternalForce('k * periodicdistance(x, y, z, x0, y0, z0)^2')
force.addGlobalParameter('k', 1e5 * unit.kilocalorie_per_mole / unit.nanometer ** 2)
force.addPerParticleParameter('x0')
force.addPerParticleParameter('y0')
force.addPerParticleParameter('z0')
system.addForce(force)

We need to tell OpenMM to which atoms to apply the position restraints. We will use MDAnalysis to identify these atoms.

Create an AtomGroup containing the "N", "CA", and "C" atoms of the peptide bachbone from the universe with hydrogen atoms we built earlier.

In [ ]:
backbone = u_hydrogen.select_atoms('name N CA C')

Adding the a particle to the pool of position restraints is done with the `force.addParticle` method. The first argument to that method is the index of the atom in the overall structure (starting from 0), the second argument is a list of the x, y, and z coordinate for the atom in nanometers.

Add the backbone atoms we just selected to the position restraints.

In [ ]:
for atom in backbone:
    position = [pos / 10 for pos in atom.position]
    force.addParticle(int(atom.ix), position)

Now we can run the energy minimisation.

In [ ]:
# Create the simulation object
simulation = app.Simulation(fixer.topology, system, integrator)
# Set the position for each atom
simulation.context.setPositions(fixer.positions)
# Finally minimise the energy of the system
simulation.minimizeEnergy()

## Check the final backbone with MDAnalysis

We have a final structure. Load it into MDAnalysis and visualise it with NGLview.

In [ ]:
minimized = mda.Universe(simulation)

In [ ]:
v = nv.show_mdanalysis(minimized)
v.add_ball_and_stick()
v

Finally, check that the dihedral angles are the ones we expect: -60°, -50°, and 180° for the phi, psi and omega angles, respectively.

In [ ]:
[ag.dihedral.value() for ag in minimized.residues.phi_selections() if ag is not None]

In [ ]:
[ag.dihedral.value() for ag in minimized.residues.psi_selections() if ag is not None]

In [ ]:
[ag.dihedral.value() for ag in minimized.residues.omega_selections() if ag is not None]

## Going further

* Progressively relax the position restraints on the backbone during the energy minimisation. We used very strong position restraints on the backbone when minimising the energy of the peptide. This is needed because the unphysical structure yields very high energies. As the side chain are better placed after the first round of energy minimisation, it should be possible to run subsequent energy minimisation steps with weaker position restraints to obtain a more realistic backbone.
* Insert the peptide in a lipid bilayer. The peptide we built is called WALP, it is an artificial peptide that have been designed to study how protein behave in lipid bilayers. Get a lipid bilayer from the [NMRlipids project](https://github.com/NMRLipids/Databank), insert the peptide perpendicular to the lipid bilayer, and remove the lipid and solvent molecules that overlap with the peptide.